In [1]:
!export PYDEVD_DISABLE_FILE_VALIDATION=1

In [2]:
!git clone https://github.com/feed7362/Clotex_api
%cd Clotex_api/backend
!uv sync
%cd ..


Cloning into 'Clotex_api'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 159 (delta 49), reused 146 (delta 42), pack-reused 0 (from 0)
Receiving objects: 100% (159/159), 1.29 MiB | 26.89 MiB/s, done.
Resolving deltas: 100% (49/49), done.
/content/Clotex_api/backend
Using CPython 3.12.12 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
Resolved 111 packages in 29ms
Prepared 103 packages in 1m 39s
Installed 103 packages in 1.26s
 + absl-py==2.3.1
 + accelerate==1.11.0
 + annotated-doc==0.0.3
 + annotated-types==0.7.0
 + anyio==4.11.0
 + astunparse==1.6.3
 + certifi==2025.10.5
 + charset-normalizer==3.4.4
 + click==8.3.0
 + coloredlogs==15.0.1
 + fastapi==0.120.4
 + filelock==3.20.0
 + flatbuffers==25.9.23
 + fsspec==2025.10.0
 + gast==0.6.0
 + google-pasta==0.2.0
 + grpcio==1.76.0
 + h11==0.16.0
 + h5py==3.15.1
 + hf-xet==1.2.0
 + httptools==0.7.1
 + hugging

In [3]:
!pip install flask requests pyngrok

import subprocess, time, threading
from flask import Flask, send_from_directory, request
import requests
from pyngrok import ngrok

# ---------- 1️⃣ Start FastAPI backend ----------
backend_process = subprocess.Popen([
    "uv", "run", "uvicorn", "main:app",
    "--host", "0.0.0.0", "--port", "8000"
], cwd="./backend",     stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1)

time.sleep(150)  # Give backend time to boot
def stream_backend_logs(proc):
    print("📜 Backend logs (live):")
    for line in iter(proc.stdout.readline, ''):
        if not line:
            break
        print("🧠", line.strip())
    print("⚠️ Backend log stream ended.")
threading.Thread(target=stream_backend_logs, daemon=True, args=(backend_process,)).start()

# ---------- 2️⃣ Flask static + API proxy ----------
FRONTEND_DIR = "./frontend"
BACKEND_URL = "http://127.0.0.1:8000"

app = Flask(__name__, static_folder=FRONTEND_DIR)


@app.route("/api/<path:path>", methods=["GET","POST","PUT","DELETE"])
def api_proxy(path):
    url = f"{BACKEND_URL}/api/{path}"
    resp = requests.request(
        method=request.method,
        url=url,
        headers={k: v for k, v in request.headers.items() if k.lower() != "host"},
        data=request.get_data(),
        cookies=request.cookies,
        allow_redirects=False)
    return (resp.content, resp.status_code, resp.headers.items())

@app.route("/")
def index():
    return send_from_directory(FRONTEND_DIR, "index.html")

@app.route("/<path:path>")
def static_proxy(path):
    return send_from_directory(FRONTEND_DIR, path)

# ---------- 3️⃣ Run Flask in background ----------
def run_flask():
    app.run(host="0.0.0.0", port=3000, debug=False, use_reloader=False)

threading.Thread(target=run_flask, daemon=True).start()
time.sleep(3)

# ---------- 4️⃣ Expose via ngrok ----------
ngrok.set_auth_token("353y4cZpz3P1rrhwPjMxpQ0pDoV_3Dh8JvYy5EDxyFLuVzRuv")
public_url = ngrok.connect(3000, bind_tls=True)
print("🚀 Public URL:", public_url)
print("✅ Flask frontend + FastAPI backend are both live")

try:
    resp = requests.get(f"{BACKEND_URL}/api/health/live", timeout=5)
    print("Backend health:", resp.status_code, resp.text)
except Exception as e:
    print("⚠️ Backend not reachable:", e)

📜 Backend logs (live):
🧠 2025-11-05 15:15:21.189927: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
🧠 To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
🧠 INFO:     Started server process [3100]
🧠 INFO:     Waiting for application startup.
🧠 2025-11-05 15:15:35,424 [INFO] [root] 🧠 Initializing application lifespan context...
🧠 /content/Clotex_api/backend/.venv/lib/python3.12/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/d

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:3000
 * Running on http://172.28.0.12:3000
INFO:werkzeug:Press CTRL+C to quit


🚀 Public URL: NgrokTunnel: "https://hirstie-brianne-retributively.ngrok-free.dev" -> "http://localhost:3000"
✅ Flask frontend + FastAPI backend are both live
Backend health: 200 {"status":"ok","details":null}


In [33]:
ngrok.kill()
!lsof -i:8000

COMMAND  PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python3  178 root   57u  IPv4 289109      0t0  TCP localhost:46814->localhost:8000 (CLOSE_WAIT)
uvicorn 9759 root   53u  IPv4 276003      0t0  TCP *:8000 (LISTEN)


In [4]:
!pkill -f uvicorn


In [4]:
!curl http://127.0.0.1:8000/api/health/live


INFO:werkzeug:127.0.0.1 - - [05/Nov/2025 15:18:03] "POST /api/raw_image/process HTTP/1.1" 200 -


🧠 2025-11-05 15:18:03,016 [INFO] [image_processing.segment] [segment] SAM2 returned 2 mask(s)
🧠 2025-11-05 15:18:03,036 [INFO] [debug.save] [debug.save] Saved mask_1 → debug_results/seg/mask_1.png
🧠 2025-11-05 15:18:03,054 [INFO] [debug.save] [debug.save] Saved mask_2 → debug_results/seg/mask_2.png
🧠 2025-11-05 15:18:03,054 [INFO] [image_processing.segment] ✅ Created 2 segmented images in 9.89s
🧠 2025-11-05 15:18:03,054 [INFO] [image_processing] [12.jpg] Step:segment ok
🧠 2025-11-05 15:18:03,054 [INFO] [image_processing] [12.jpg] Step:classify started
🧠 2025-11-05 15:18:03,054 [INFO] [image_processing.classify] [classify] Starting classification on 2 mask(s), threshold=0.5
🧠 2025-11-05 15:18:03,103 [INFO] [image_processing.classify] [classify] Mask 1: score=0.1783, time=0.049s
🧠 2025-11-05 15:18:03,116 [INFO] [image_processing.classify] [classify] Mask 2: score=0.1006, time=0.013s
🧠 2025-11-05 15:18:03,116 [INFO] [image_processing.classify] [classify] Completed: 0 / 2 mask(s) retained
